# How to migrate from legacy LangChain agents to LangGraph

Here we focus on how to move from legacy LangChain agents to LangGraph agents.
LangChain agents (the
[AgentExecutor](https://api.python.langchain.com/en/latest/agents/langchain.agents.agent.AgentExecutor.html#langchain.agents.agent.AgentExecutor)
in particular) have multiple configuration parameters. In this notebook we will
show how those parameters map to the LangGraph
[react agent executor](https://langchain-ai.github.io/langgraph/reference/prebuilt/#create_react_agent).

#### Prerequisites

This how-to guide uses Claude (model: "claude-3-haiku-20240307") as the LLM. Set
your Anthropic API key to run.


In [1]:
// process.env.ANTHROPIC_API_KEY = "sk-...";

// Optional, add tracing in LangSmith
// process.env.LANGCHAIN_API_KEY = "ls...";
process.env.LANGCHAIN_CALLBACKS_BACKGROUND = "true";
process.env.LANGCHAIN_TRACING_V2 = "true";
process.env.LANGCHAIN_PROJECT = "How to migrate: LangGraphJS";


How to migrate: LangGraphJS


## Basic Usage

For basic creation and usage of a tool-calling ReAct-style agent, the
functionality is the same. First, let's define a model and tool(s), then we'll
use those to create an agent.


In [2]:
import { DynamicStructuredTool } from "@langchain/core/tools";
import { z } from "zod";
import { ChatAnthropic } from "@langchain/anthropic";

const llm = new ChatAnthropic({ model: "claude-3-haiku-20240307" });

const magicTool = new DynamicStructuredTool({
  name: "magic_function",
  description: "Applies a magic function to an input.",
  schema: z.object({
    input: z.number(),
  }),
  func: async ({ input }: { input: number }) => {
    return `${input + 2}`;
  },
});

const tools = [magicTool];

const query = "what is the value of magic_function(3)?";


For the LangChain
[AgentExecutor](https://api.python.langchain.com/en/latest/agents/langchain.agents.agent.AgentExecutor.html#langchain.agents.agent.AgentExecutor),
we define a prompt with a placeholder for the agent's scratchpad. The agent can
be invoked as follows:


In [8]:
import {
  ChatPromptTemplate,
  MessagesPlaceholder,
} from "@langchain/core/prompts";
import { createToolCallingAgent } from "langchain/agents";
import { AgentExecutor } from "langchain/agents";

const prompt = ChatPromptTemplate.fromMessages([
  ["system", "You are a helpful assistant"],
  ["human", `{input}`],
  new MessagesPlaceholder("agent_scratchpad"),
]);

const agent = createToolCallingAgent({ llm: llm, tools: tools, prompt });
const agentExecutor = new AgentExecutor({ agent, tools: tools });

await agentExecutor.invoke({ input: query });


{
  input: 'what is the value of magic_function(3)?',
  output: 'The magic_function takes an input number and returns a value. When we pass in the value 3, the function returns 5.\n' +
    '\n' +
    'So the value of magic_function(3) is 5.'
}


LangGraph's
[react agent executor](https://langchain-ai.github.io/langgraph/reference/prebuilt/#create_react_agent)
manages a state that is defined by a list of messages. It will continue to
process the list until there are no tool calls in the agent's output. To kick it
off, we input a list of messages. The output will contain the entire state of
the graph-- in this case, the conversation history.


In [15]:
import { createReactAgent } from "@langchain/langgraph/prebuilt";
import { HumanMessage } from "@langchain/core/messages";

const app = createReactAgent(llm, tools);

let agentOutput = await app.invoke({ messages: [new HumanMessage(query)] });


Skipping write for channel branch:agent:shouldContinue:undefined which has no readers


{
  messages: [
    HumanMessage {
      lc_serializable: true,
      lc_kwargs: [Object],
      lc_namespace: [Array],
      content: 'what is the value of magic_function(3)?',
      name: undefined,
      additional_kwargs: {},
      response_metadata: {}
    },
    AIMessage {
      lc_serializable: true,
      lc_kwargs: [Object],
      lc_namespace: [Array],
      content: [Array],
      name: undefined,
      additional_kwargs: [Object],
      response_metadata: [Object],
      tool_calls: [Array],
      invalid_tool_calls: []
    },
    ToolMessage {
      lc_serializable: true,
      lc_kwargs: [Object],
      lc_namespace: [Array],
      content: '5',
      name: 'magic_function',
      additional_kwargs: {},
      response_metadata: {},
      tool_call_id: 'toolu_01Bso84yaQRUw8CBf2qzimwv'
    },
    AIMessage {
      lc_serializable: true,
      lc_kwargs: [Object],
      lc_namespace: [Array],
      content: 'The magic_function takes a numeric input and returns a value 2 gre

In [18]:
const messageHistory = agentOutput.messages;
const newQuery = "Pardon?";

agentOutput = await app.invoke({
  messages: [...messageHistory, new HumanMessage(newQuery)],
});


Skipping write for channel branch:agent:shouldContinue:undefined which has no readers


{
  messages: [
    HumanMessage {
      lc_serializable: true,
      lc_kwargs: [Object],
      lc_namespace: [Array],
      content: 'what is the value of magic_function(3)?',
      name: undefined,
      additional_kwargs: {},
      response_metadata: {}
    },
    AIMessage {
      lc_serializable: true,
      lc_kwargs: [Object],
      lc_namespace: [Array],
      content: [Array],
      name: undefined,
      additional_kwargs: [Object],
      response_metadata: [Object],
      tool_calls: [Array],
      invalid_tool_calls: []
    },
    ToolMessage {
      lc_serializable: true,
      lc_kwargs: [Object],
      lc_namespace: [Array],
      content: '5',
      name: 'magic_function',
      additional_kwargs: {},
      response_metadata: {},
      tool_call_id: 'toolu_01Bso84yaQRUw8CBf2qzimwv'
    },
    AIMessage {
      lc_serializable: true,
      lc_kwargs: [Object],
      lc_namespace: [Array],
      content: 'The magic_function takes a numeric input and returns a value 2 gre

## Prompt Templates

With legacy LangChain agents you have to pass in a prompt template. You can use
this to control the agent.

With LangGraph
[react agent executor](https://langchain-ai.github.io/langgraph/reference/prebuilt/#create_react_agent),
by default there is no prompt. You can achieve similar control over the agent in
a few ways:

1. Pass in a system message as input
2. Initialize the agent with a system message
3. Initialize the agent with a function to transform messages before passing to
   the model.

Let's take a look at all of these below. We will pass in custom instructions to
get the agent to respond in Spanish.

First up, using AgentExecutor:


In [22]:
const spanishPrompt = ChatPromptTemplate.fromMessages([
  ["system", "You are a helpful assistant. Respond only in Spanish."],
  ["human", `{input}`],
  new MessagesPlaceholder("agent_scratchpad"),
]);

const spanishAgent = createToolCallingAgent({
  llm: llm,
  tools: tools,
  prompt: spanishPrompt,
});
const spanishAgentExecutor = new AgentExecutor({
  agent: spanishAgent,
  tools: tools,
});

await spanishAgentExecutor.invoke({ input: query });


{
  input: 'what is the value of magic_function(3)?',
  output: 'El valor de magic_function(3) es 5.'
}


Now, let's pass a custom system message to
[react agent executor](https://langchain-ai.github.io/langgraph/reference/prebuilt/#create_react_agent).
This can either be a string or a LangChain SystemMessage.


In [25]:
import { SystemMessage } from "@langchain/core/messages";

const systemMessage = "You are a helpful assistant. Respond only in Spanish.";

// This could also be a SystemMessage object
// const systemMessage = new SystemMessage("You are a helpful assistant. Respond only in Spanish.");

const appWithSystemMessage = createReactAgent(llm, tools, systemMessage);

agentOutput = await appWithSystemMessage.invoke({
  messages: [new HumanMessage(query)],
});


Skipping write for channel branch:agent:shouldContinue:undefined which has no readers


{
  messages: [
    HumanMessage {
      lc_serializable: true,
      lc_kwargs: [Object],
      lc_namespace: [Array],
      content: 'what is the value of magic_function(3)?',
      name: undefined,
      additional_kwargs: {},
      response_metadata: {}
    },
    AIMessage {
      lc_serializable: true,
      lc_kwargs: [Object],
      lc_namespace: [Array],
      content: [Array],
      name: undefined,
      additional_kwargs: [Object],
      response_metadata: [Object],
      tool_calls: [Array],
      invalid_tool_calls: []
    },
    ToolMessage {
      lc_serializable: true,
      lc_kwargs: [Object],
      lc_namespace: [Array],
      content: '5',
      name: 'magic_function',
      additional_kwargs: {},
      response_metadata: {},
      tool_call_id: 'toolu_01K3hHzK9KiksF87sDRaJxYW'
    },
    AIMessage {
      lc_serializable: true,
      lc_kwargs: [Object],
      lc_namespace: [Array],
      content: 'El valor de magic_function(3) es 5.',
      name: undefined,
     

We can also pass in an arbitrary function. This function should take in a list
of messages and output a list of messages. We can do all types of arbitrary
formatting of messages here. In this cases, let's just add a SystemMessage to
the start of the list of messages.


In [38]:
import { BaseMessage, SystemMessage } from "@langchain/core/messages";

const modifyMessages = (messages: BaseMessage[]) => {
  return [
    new SystemMessage("You are a helpful assistant. Respond only in Spanish."),
    ...messages,
    new HumanMessage("Also say 'Pandamonium!' after the answer."),
  ];
};

const appWithMessagesModifier = createReactAgent(llm, tools, modifyMessages);

agentOutput = await appWithMessagesModifier.invoke({
  messages: [new HumanMessage(query)],
});

console.log({
  input: query,
  output: agentOutput.messages[agentOutput.messages.length - 1].content,
});


Skipping write for channel branch:agent:shouldContinue:undefined which has no readers


{
  input: 'what is the value of magic_function(3)?',
  output: 'El resultado de magic_function(3) es 5. ¡Pandamonium!'
}


## Memory

With LangChain's
[AgentExecutor](https://api.python.langchain.com/en/latest/agents/langchain.agents.agent.AgentExecutor.html#langchain.agents.agent.AgentExecutor.iter),
you could add chat
[Memory](https://api.python.langchain.com/en/latest/agents/langchain.agents.agent.AgentExecutor.html#langchain.agents.agent.AgentExecutor.memory)
so it can engage in a multi-turn conversation.


In [45]:
import { ChatMessageHistory } from "@langchain/community/stores/message/in_memory";
import { RunnableWithMessageHistory } from "@langchain/core/runnables";

const memory = new ChatMessageHistory();
const agentExecutorWithMemory = new RunnableWithMessageHistory({
  runnable: agentExecutor,
  getMessageHistory: () => memory,
  inputMessagesKey: "input",
  historyMessagesKey: "chat_history",
});

const config = { configurable: { sessionId: "test-session" } };

agentOutput = await agentExecutorWithMemory.invoke(
  { input: "Hi, I'm polly! What's the output of magic_function of 3?" },
  config
);
console.log(agentOutput.output);
agentOutput = await agentExecutorWithMemory.invoke(
  { input: "Remember my name?" },
  config
);
console.log("---");
console.log(agentOutput.output);
console.log("---");
agentOutput = await agentExecutorWithMemory.invoke(
  { input: "what was that output again?" },
  config
);
console.log(agentOutput.output);


The output of the magic_function with an input of 3 is 5.
---
I'm afraid I don't actually have a way to remember your name, as I'm an AI assistant without a persistent memory. I don't have a way to store or recall specific details about individual users I've interacted with. I'm happy to continue our conversation, but I won't be able to reference any previous information about you or remember your name specifically. Please feel free to introduce yourself again if you'd like me to know your name.
---
I'm sorry, but I don't have any previous output to reference. Could you please provide more context about what you are asking about? I'd be happy to try to assist you, but I need more information about the specific request or output you are referring to.


#### In LangGraph

Memory is just
[persistence](https://langchain-ai.github.io/langgraph/how-tos/persistence/),
aka
[checkpointing](https://langchain-ai.github.io/langgraph/reference/checkpoints/).

Add a `checkpointer` to the agent and you get chat memory for free.


In [55]:
import { MemorySaver } from "@langchain/langgraph";

const memory = new MemorySaver();
const appWithMemory = createReactAgent(llm, tools, systemMessage, memory);

const config = { configurable: { thread_id: "test-thread" } };

agentOutput = await appWithMemory.invoke(
  {
    messages: [
      new HumanMessage(
        "Hi, I'm polly! What's the output of magic_function of 3?"
      ),
    ],
  },
  config
);
console.log(agentOutput.messages[agentOutput.messages.length - 1].content);
console.log("---");
agentOutput = await appWithMemory.invoke(
  { messages: [new HumanMessage("Remember my name?")] },
  config
);
console.log(agentOutput.messages[agentOutput.messages.length - 1].content);
console.log("---");
agentOutput = await appWithMemory.invoke(
  { messages: [new HumanMessage("what was that output again?")] },
  config
);
console.log(agentOutput.messages[agentOutput.messages.length - 1].content);


Skipping write for channel branch:agent:shouldContinue:undefined which has no readers


La función "magic_function" toma un número como entrada y devuelve un número diferente como salida. Cuando se le pasa el número 3, la función devuelve 5 como resultado.
---


Skipping write for channel branch:agent:shouldContinue:undefined which has no readers


Claro que recuerdo tu nombre, Polly. Es un placer hablar contigo de nuevo.
---


Skipping write for channel branch:agent:shouldContinue:undefined which has no readers


El resultado anterior de la función "magic_function" aplicada al número 3 es 5.


## Iterating through steps

With LangChain's
[AgentExecutor](https://api.python.langchain.com/en/latest/agents/langchain.agents.agent.AgentExecutor.html#langchain.agents.agent.AgentExecutor.iter),
you could iterate over the steps using the
[stream](https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.base.Runnable.html#langchain_core.runnables.base.Runnable.stream)
(or async `astream`) methods or the
[iter](https://api.python.langchain.com/en/latest/agents/langchain.agents.agent.AgentExecutor.html#langchain.agents.agent.AgentExecutor.iter)
method. LangGraph supports stepwise iteration using
[stream](https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.base.Runnable.html#langchain_core.runnables.base.Runnable.stream)


In [57]:
for await (const step of await agentExecutor.stream({ input: query })) {
  console.log(step);
}


{ intermediateSteps: [ { action: [Object], observation: '5' } ] }
{
  output: 'The magic_function takes an input number and returns a value that is 2 more than the input. So the value of magic_function(3) is 5.'
}


#### In LangGraph

In LangGraph, things are handled natively using
[stream](https://langchain-ai.github.io/langgraph/reference/graphs/#langgraph.graph.graph.CompiledGraph.stream)
or the asynchronous `astream` method.


In [58]:
for await (const step of await app.stream(
  { messages: [new HumanMessage(query)] },
  { streamMode: "updates" }
)) {
  console.log(step);
}


{ agent: { messages: [ [AIMessage] ] } }
{ tools: { messages: [ [ToolMessage] ] } }


Skipping write for channel branch:agent:shouldContinue:undefined which has no readers


{ agent: { messages: [ [AIMessage] ] } }


## `return_intermediate_steps`

Setting this parameter on AgentExecutor allows users to access
intermediate_steps, which pairs agent actions (e.g., tool invocations) with
their outcomes.


In [65]:
const agentExecutorWithIntermediateSteps = new AgentExecutor({
  agent: agent,
  tools: tools,
  returnIntermediateSteps: true,
});

const result = await agentExecutorWithIntermediateSteps.invoke({
  input: query,
});

console.log(result.intermediateSteps);


[
  {
    action: {
      tool: 'magic_function',
      toolInput: [Object],
      toolCallId: 'toolu_017S5HcJGFrSs3uS6ShB3ftH',
      log: 'Invoking "magic_function" with {"input":3}\n' +
        '[{"type":"tool_use","id":"toolu_017S5HcJGFrSs3uS6ShB3ftH","name":"magic_function","input":{"input":3}}]',
      messageLog: [Array]
    },
    observation: '5'
  }
]


By default the
[react agent executor](https://langchain-ai.github.io/langgraph/reference/prebuilt/#create_react_agent)
in LangGraph appends all messages to the central state. Therefore, it is easy to
see any intermediate steps by just looking at the full state.


In [66]:
messages = await app.invoke({ messages: [new HumanMessage(query)] });

console.log(messages);


Skipping write for channel branch:agent:shouldContinue:undefined which has no readers


{
  messages: [
    HumanMessage {
      lc_serializable: true,
      lc_kwargs: [Object],
      lc_namespace: [Array],
      content: 'what is the value of magic_function(3)?',
      name: undefined,
      additional_kwargs: {},
      response_metadata: {}
    },
    AIMessage {
      lc_serializable: true,
      lc_kwargs: [Object],
      lc_namespace: [Array],
      content: [Array],
      name: undefined,
      additional_kwargs: [Object],
      response_metadata: [Object],
      tool_calls: [Array],
      invalid_tool_calls: []
    },
    ToolMessage {
      lc_serializable: true,
      lc_kwargs: [Object],
      lc_namespace: [Array],
      content: '5',
      name: 'magic_function',
      additional_kwargs: {},
      response_metadata: {},
      tool_call_id: 'toolu_01LkD1kvFBXQvRaR5sj461q3'
    },
    AIMessage {
      lc_serializable: true,
      lc_kwargs: [Object],
      lc_namespace: [Array],
      content: 'The magic_function takes a single numeric input parameter and retu

## `max_iterations`

`AgentExecutor` implements a `max_iterations` parameter, whereas this is
controlled via `recursion_limit` in LangGraph.

Note that in AgentExecutor, an "iteration" includes a full turn of tool
invocation and execution. In LangGraph, each step contributes to the recursion
limit, so we will need to multiply by two (and add one) to get equivalent
results.

If the recursion limit is reached, LangGraph raises a specific exception type,
that we can catch and manage similarly to AgentExecutor.


In [74]:
const badMagicTool = new DynamicStructuredTool({
  name: "magic_function",
  description: "Applies a magic function to an input.",
  schema: z.object({
    input: z.string(),
  }),
  func: async ({ input }) => {
    return "Sorry, there was an error. Please try again.";
  },
});

const badTools = [badMagicTool];

const spanishAgentExecutorWithMaxIterations = new AgentExecutor({
  agent: createToolCallingAgent({
    llm: llm,
    tools: badTools,
    prompt: spanishPrompt,
  }),
  tools: badTools,
  verbose: true,
  maxIterations: 2,
});

await spanishAgentExecutorWithMaxIterations.invoke({ input: query });


[chain/start] [1:chain:AgentExecutor] Entering Chain run with input: {
  "input": "what is the value of magic_function(3)?"
}
[chain/start] [1:chain:AgentExecutor > 2:chain:ToolCallingAgent] Entering Chain run with input: {
  "input": "what is the value of magic_function(3)?",
  "steps": []
}
[chain/start] [1:chain:AgentExecutor > 2:chain:ToolCallingAgent > 3:chain:RunnableAssign] Entering Chain run with input: {
  "input": ""
}
[chain/start] [1:chain:AgentExecutor > 2:chain:ToolCallingAgent > 3:chain:RunnableAssign > 4:chain:RunnableMap] Entering Chain run with input: {
  "input": ""
}
[chain/start] [1:chain:AgentExecutor > 2:chain:ToolCallingAgent > 3:chain:RunnableAssign > 4:chain:RunnableMap > 5:chain:RunnableLambda] Entering Chain run with input: {
  "input": ""
}
[chain/end] [1:chain:AgentExecutor > 2:chain:ToolCallingAgent > 3:chain:RunnableAssign > 4:chain:RunnableMap > 5:chain:RunnableLambda] s] Exiting Chain run with output: {
  "output": []
}
[chain/end] [1:chain:AgentExecut

In [ ]:
import { GraphRecursionError } from "@langchain/langgraph";

const RECURSION_LIMIT = 2 * 2 + 1;

const appWithBadTools = createReactAgent(llm, badTools);

try {
  await appWithBadTools.invoke({ messages: [new HumanMessage(query)] }, {recursionLimit: RECURSION_LIMIT});
} catch (e) {
    if (e instanceof GraphRecursionError) {
        console.log("Recursion limit reached.");
    } else {
        throw e;
    }
}


Recursion limit reached.


UncaughtException: Error: Unexpected pending rebuildTimer
    at sys.setTimeout (/usr/local/lib/node_modules/tslab/dist/converter.js:111:19)
    at scheduleProgramUpdate (/usr/local/lib/node_modules/tslab/node_modules/@tslab/typescript-for-tslab/lib/typescript.js:122735:35)
    at onSourceFileChange (/usr/local/lib/node_modules/tslab/node_modules/@tslab/typescript-for-tslab/lib/typescript.js:122876:7)
    at /usr/local/lib/node_modules/tslab/node_modules/@tslab/typescript-for-tslab/lib/typescript.js:122868:56
    at cb (/usr/local/lib/node_modules/tslab/dist/converter.js:184:13)
    at /usr/local/lib/node_modules/tslab/node_modules/@tslab/typescript-for-tslab/lib/typescript.js:5798:9
    at /usr/local/lib/node_modules/tslab/node_modules/@tslab/typescript-for-tslab/lib/typescript.js:5560:101
    at Array.forEach (<anonymous>)
    at /usr/local/lib/node_modules/tslab/node_modules/@tslab/typescript-for-tslab/lib/typescript.js:5560:85
    at FSWatcher.callbackChangingToMissingFileSystemEnt